# PERCEPTRON CLASSIFIER

In [1]:
!pip install torch torchvision scikit-learn

# Libraries import

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import random
from torch.nn import Softmax

# EXAMPLE: IRIS DATASET

![](https://upload.wikimedia.org/wikipedia/commons/4/41/Iris_versicolor_3.jpg)


Let's load the dataset from the Scikit learn hub!

In [3]:
# Load the Iris dataset
iris = load_iris()
X_raw = iris.data
y = iris.target

In [4]:
iris["DESCR"]

'.. _iris_dataset:\n\nIris plants dataset\n--------------------\n\n**Data Set Characteristics:**\n\n:Number of Instances: 150 (50 in each of three classes)\n:Number of Attributes: 4 numeric, predictive attributes and the class\n:Attribute Information:\n    - sepal length in cm\n    - sepal width in cm\n    - petal length in cm\n    - petal width in cm\n    - class:\n            - Iris-Setosa\n            - Iris-Versicolour\n            - Iris-Virginica\n\n:Summary Statistics:\n\n============== ==== ==== ======= ===== ====================\n                Min  Max   Mean    SD   Class Correlation\n============== ==== ==== ======= ===== ====================\nsepal length:   4.3  7.9   5.84   0.83    0.7826\nsepal width:    2.0  4.4   3.05   0.43   -0.4194\npetal length:   1.0  6.9   3.76   1.76    0.9490  (high!)\npetal width:    0.1  2.5   1.20   0.76    0.9565  (high!)\n============== ==== ==== ======= ===== ====================\n\n:Missing Attribute Values: None\n:Class Distribution: 

For example...

In [5]:
X_raw[0], y[0]

(array([5.1, 3.5, 1.4, 0.2]), np.int64(0))

In [6]:
feature_names = iris["feature_names"]
target_names = iris["target_names"]
example = X_raw[0].tolist()
example_gt = y[0]
for ix, val in enumerate(example):
    print(f"{feature_names[ix].capitalize()}: {val}", end = ", " if ix != len(feature_names) - 1 else "\n")
print(f"Target: {target_names[example_gt]}")

Sepal length (cm): 5.1, Sepal width (cm): 3.5, Petal length (cm): 1.4, Petal width (cm): 0.2
Target: setosa


## PREPROCESSING

In [7]:
# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X_raw)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

## DATASET SPLITTING

In [8]:
# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## SETTING THE TORCH DATASET

In [9]:
# Create DataLoaders with minibatching
batch_size = 16

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

## SET THE CLASSIFIER

In [10]:
# Define the single-layer classifier
class IrisClassifier(nn.Module):
    def __init__(self):
        super(IrisClassifier, self).__init__()
        self.fc = nn.Linear(4, 3) # input features -> output classes

    def forward(self, x):
        return self.fc(x)

# START THE TRAINING

In [11]:
# Initialize model, loss function, and optimizer
model = IrisClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop with minibatching
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_X, batch_y in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation accuracy
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for val_X, val_y in val_loader:
            val_outputs = model(val_X)
            _, predicted = torch.max(val_outputs, 1)
            all_preds.extend(predicted.tolist())
            all_labels.extend(val_y.tolist())
    val_acc = accuracy_score(all_labels, all_preds)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Val Accuracy: {val_acc:.2f}")


Epoch [10/100], Loss: 3.0372, Val Accuracy: 0.93
Epoch [20/100], Loss: 2.2570, Val Accuracy: 0.97
Epoch [30/100], Loss: 1.8357, Val Accuracy: 0.97
Epoch [40/100], Loss: 1.5723, Val Accuracy: 1.00
Epoch [50/100], Loss: 1.3674, Val Accuracy: 1.00
Epoch [60/100], Loss: 1.2147, Val Accuracy: 1.00
Epoch [70/100], Loss: 1.0864, Val Accuracy: 1.00
Epoch [80/100], Loss: 1.0325, Val Accuracy: 1.00
Epoch [90/100], Loss: 0.9256, Val Accuracy: 1.00
Epoch [100/100], Loss: 0.9033, Val Accuracy: 1.00


## TESTING THE MODEL

In [12]:
test_id = random.randint(0, len(val_dataset))
f"Testing val data point n.{test_id}"

'Testing val data point n.4'

In [13]:
features = X_val[test_id].unsqueeze(dim=0)
gt = y_val[test_id]
print(f"Features: {features}, gt: {gt}")

Features: tensor([[ 1.1592, -0.5924,  0.5922,  0.2641]]), gt: 1


In [14]:
with torch.no_grad():
    output = model(features)
output

output.argmax()

tensor(1)

In [15]:
reg_out = Softmax(dim=-1)(output)
reg_out

tensor([[2.3230e-04, 8.1010e-01, 1.8966e-01]])

In [16]:
output.argmax()

tensor(1)

In [17]:
raw_features = scaler.inverse_transform(features.numpy())[0].tolist()
feature_information = ", ".join([f"{k:.2f}: {v}" for (k, v) in zip(raw_features, feature_names)])
f"How is classified a data point with {feature_information}? Classification: {target_names[output.argmax().item()]} with prob. of {reg_out.squeeze()[output.argmax().item()]*100:.2f}%"

'How is classified a data point with 6.80: sepal length (cm), 2.80: sepal width (cm), 4.80: petal length (cm), 1.40: petal width (cm)? Classification: versicolor with prob. of 81.01%'

In [18]:
f"Is it a correctly classified example? {'Yes' if output.argmax().item() == gt else 'No'}"

'Is it a correctly classified example? Yes'